# Smart Factory Energy Prediction – Final Solution
This notebook contains the **complete code** for:
1. Data loading & cleaning
2. Exploratory analysis & feature selection
3. Train/Val/Test split
4. Model training & hyper‑parameter tuning (Ridge, Lasso, RandomForest, XGBoost, PyTorch MLP)
5. Final evaluation & explainability
6. Model persistence

> Generated automatically on 2025‑05‑09

In [ ]:
import pandas as pd, numpy as np, warnings, os, random, math, joblib, matplotlib.pyplot as plt
warnings.filterwarnings('ignore')
SEED = 42
np.random.seed(SEED); random.seed(SEED)


In [ ]:
# ------------ Data Load ------------
df = pd.read_csv('data.csv')

# Target to numeric
df['equipment_energy_consumption'] = pd.to_numeric(df['equipment_energy_consumption'], errors='coerce')
df = df.dropna(subset=['equipment_energy_consumption'])

# Drop timestamp
if 'timestamp' in df.columns:
    df = df.drop(columns=['timestamp'])

# Convert object columns
for col in df.select_dtypes('object').columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Fill missing with median
df = df.fillna(df.median(numeric_only=True))

print('Dataset shape:', df.shape)
df.head()


In [ ]:
from sklearn.feature_selection import mutual_info_regression

# Remove random variables
for col in ['random_variable1','random_variable2']:
    if col in df.columns: df = df.drop(columns=[col])

# Drop highly correlated pairs (|ρ|>0.8)
corr = df.corr().abs()
upper = corr.where(np.triu(np.ones(corr.shape),k=1).astype(bool))
to_drop = [c for c in upper.columns if any(upper[c] > 0.8)]
df = df.drop(columns=to_drop)
print('Dropped (high corr):', to_drop)

# Mutual information top‑k
target = 'equipment_energy_consumption'
X_all = df.drop(columns=[target]); y_all = df[target]
mi_scores = mutual_info_regression(X_all, y_all, random_state=SEED)
mi_series = pd.Series(mi_scores, index=X_all.columns).sort_values(ascending=False)
top_k = 25
selected_feats = mi_series.head(top_k).index.tolist()
print('Selected features:', selected_feats)


In [ ]:
from sklearn.model_selection import train_test_split
X = df[selected_feats]
y = df[target].values
X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size=0.20, random_state=SEED)
X_train, X_val, y_train, y_val       = train_test_split(X_trainval, y_trainval, test_size=0.125, random_state=SEED)
print(f'Train {X_train.shape}, Val {X_val.shape}, Test {X_test.shape}')


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import xgboost as xgb, torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

def rmse(a,b): return mean_squared_error(a,b,squared=False)

results = {}

# ---- Ridge
ridge_pipe = Pipeline([('scale',StandardScaler()),('ridge',Ridge())])
ridge_grid = {'ridge__alpha':[0.01,0.1,1,10,100]}
ridge_cv = GridSearchCV(ridge_pipe, ridge_grid, cv=5, scoring='neg_root_mean_squared_error', n_jobs=-1)
ridge_cv.fit(X_train, y_train)
results['Ridge'] = ridge_cv.best_estimator_

# ---- Lasso
lasso_pipe = Pipeline([('scale',StandardScaler()),('lasso',Lasso(max_iter=5000))])
lasso_grid = {'lasso__alpha':[0.001,0.01,0.1,1,10]}
lasso_cv = GridSearchCV(lasso_pipe, lasso_grid, cv=5, scoring='neg_root_mean_squared_error', n_jobs=-1)
lasso_cv.fit(X_train, y_train)
results['Lasso'] = lasso_cv.best_estimator_

# ---- Random Forest
rf_grid = {'n_estimators':[300,600], 'max_depth':[None,20,40], 'min_samples_leaf':[1,2]}
rf_cv = RandomizedSearchCV(RandomForestRegressor(random_state=SEED,n_jobs=-1),
                           rf_grid, n_iter=12, cv=3,
                           scoring='neg_root_mean_squared_error', random_state=SEED, n_jobs=-1)
rf_cv.fit(X_train, y_train)
results['RandomForest'] = rf_cv.best_estimator_

# ---- XGBoost
xgb_grid = {'learning_rate':[0.01,0.05,0.1],
            'max_depth':[3,5,7],
            'subsample':[0.7,1.0],
            'colsample_bytree':[0.7,1.0],
            'n_estimators':[400,800]}
xgb_cv = RandomizedSearchCV(xgb.XGBRegressor(objective='reg:squarederror', random_state=SEED, n_jobs=-1),
                            xgb_grid, n_iter=20, cv=3,
                            scoring='neg_root_mean_squared_error', random_state=SEED, n_jobs=-1)
xgb_cv.fit(X_train, y_train, eval_set=[(X_val,y_val)], verbose=False)
results['XGBoost'] = xgb_cv.best_estimator_

# ---- PyTorch MLP
class MLP(nn.Module):
    def __init__(self,d_in):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(d_in,128), nn.BatchNorm1d(128), nn.ReLU(), nn.Dropout(0.25),
            nn.Linear(128,64),  nn.BatchNorm1d(64), nn.ReLU(), nn.Dropout(0.25),
            nn.Linear(64,1))
    def forward(self,x): return self.net(x)

scaler_pt = StandardScaler().fit(X_train)
def to_tensor(a): return torch.tensor(a, dtype=torch.float32)
train_ds = TensorDataset(to_tensor(scaler_pt.transform(X_train)), to_tensor(y_train).view(-1,1))
val_ds   = TensorDataset(to_tensor(scaler_pt.transform(X_val)),   to_tensor(y_val).view(-1,1))
train_dl = DataLoader(train_ds, batch_size=256, shuffle=True)
val_dl   = DataLoader(val_ds, batch_size=256, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
mlp = MLP(X_train.shape[1]).to(device)
opt = optim.AdamW(mlp.parameters(), lr=1e-3, weight_decay=1e-4)
loss_fn = nn.MSELoss()
best_val = math.inf; patience=0
for epoch in range(200):
    mlp.train()
    for xb,yb in train_dl:
        xb,yb = xb.to(device), yb.to(device)
        opt.zero_grad(); loss = loss_fn(mlp(xb), yb); loss.backward(); opt.step()
    mlp.eval()
    with torch.no_grad():
        vloss = loss_fn(mlp(to_tensor(scaler_pt.transform(X_val)).to(device)),
                        to_tensor(y_val).view(-1,1).to(device)).item()
    if vloss < best_val-1e-4:
        best_val = vloss; patience=0; torch.save(mlp.state_dict(), 'best_mlp.pt')
    else:
        patience += 1
    if patience>=20: break
mlp.load_state_dict(torch.load('best_mlp.pt'))
results['MLP'] = (mlp, scaler_pt)

# ---- Leaderboard on validation
from collections import OrderedDict
val_scores = OrderedDict()
for name, model in results.items():
    if name=='MLP':
        y_pred = model[0](to_tensor(model[1].transform(X_val)).to(device)).cpu().detach().numpy().flatten()
    else:
        y_pred = model.predict(X_val)
    val_scores[name] = rmse(y_val, y_pred)
print('Validation RMSE leaderboard:', val_scores)
best_name = min(val_scores, key=val_scores.get)
print('Best model:', best_name)


In [ ]:
from sklearn.inspection import permutation_importance
import shap

if best_name=='MLP':
    mlp, scaler_best = results['MLP']
    test_pred = mlp(to_tensor(scaler_best.transform(X_test)).to(device)).cpu().detach().numpy().flatten()
else:
    best_model = results[best_name]
    best_model.fit(pd.concat([X_train, X_val]), pd.concat([pd.Series(y_train), pd.Series(y_val)]))
    test_pred = best_model.predict(X_test)

rmse_test = rmse(y_test, test_pred)
mae_test  = mean_absolute_error(y_test, test_pred)
r2_test   = r2_score(y_test, test_pred)
print(f'Test RMSE {rmse_test:.3f}  MAE {mae_test:.3f}  R² {r2_test:.4f}')

# Explainability (tree -> SHAP)
if best_name in {'RandomForest','XGBoost'}:
    explainer = shap.TreeExplainer(best_model)
    shap_values = explainer.shap_values(X_test.sample(400, random_state=SEED))
    shap.summary_plot(shap_values, X_test.sample(400, random_state=SEED), feature_names=X_test.columns)
elif best_name in {'Ridge','Lasso'}:
    coef = pd.Series(best_model.named_steps['lasso'].coef_ if best_name=='Lasso' else best_model.named_steps['ridge'].coef_,
                     index=X_train.columns).sort_values(key=np.abs, ascending=False)
    coef.head(20).plot(kind='barh'); plt.gca().invert_yaxis(); plt.title('Top coefficients')
else:
    perm = permutation_importance(mlp, scaler_best.transform(X_test), y_test, n_repeats=15, random_state=SEED,
                                  scoring='neg_root_mean_squared_error')
    imp = pd.Series(perm.importances_mean, index=X_test.columns).sort_values(ascending=False)
    imp.head(20).plot(kind='barh'); plt.gca().invert_yaxis(); plt.title('Permutation Importance (MLP)')
plt.tight_layout()
